# Part 1

## 1.1

In [2]:
#Create text file with links (EXECUTE THE NEXT 2 CELLS ONLY ONCE)
import requests
from bs4 import BeautifulSoup
from tqdm import tqdm

anime = []

for page in tqdm(range(0, 400)):
    url = 'https://myanimelist.net/topanime.php?limit=' + str(page * 50)
    response = requests.get(url)
    
    soup = BeautifulSoup(response.text, 'html.parser')
    for tag in soup.find_all('tr'):
        links = tag.find_all('a')
        for link in links:        
            if type(link.get('id')) == str and len(link.contents[0]) > 1:
                anime.append((link.contents[0], link.get('href')) )


100%|██████████| 400/400 [05:13<00:00,  1.27it/s]


In [3]:
f = open("links.txt", "w")
for el in anime:
    f.write(str(el[1])+'\n')
f.close()
#We get 19083 rows, which implies we have less than 20000 animes

## 1.2

In [1]:
#Download the pages
import os
import requests
from bs4 import BeautifulSoup
from tqdm import tqdm
from random import randint
#multiprocessing library

#first open file from which we get links
f = open("links.txt","r")
lines = f.readlines()

for page in tqdm(range(364,382)):
    #Manage the directories
    os.system("mkdir Page"+str(page+1)) #the (page+1) is to make so we don't have the first element 0 but 1
    os.chdir("Page"+str(page+1))
    
    #These if conditions are needed to have the right indexing of the pages downloaded
    start=50*page
    
    for i in range(start,start+50):
        url=lines[i]
        check = requests.get(url)
        
        #Check if the site is blocking the download, start again when we are allowed to
        if check.status_code != 200:
            print("Last well executed page is {}".format(page-1))
            while(check.status_code != 200): #Stop while we can't download
                check = requests.get(url)
                #next line was just a try of changing the ip when the site would stop the downloads
                #in short didn't manage to implement it
                #os.system(("sudo ifconfig lo 127.0.0.{}  netmask 255.0.0.0").format(randint(0,50))) 
       
        #Save the content of the page 
        art=open("article"+str(i+1)+'.html',"wb")
        art.write(check.content)
        art.close()
    
    os.chdir("..")

 17%|█▋        | 3/18 [06:08<28:42, 114.83s/it]

Last well executed page is 366
Last well executed page is 366
Last well executed page is 366


 94%|█████████▍| 17/18 [25:00<01:28, 88.29s/it]


IndexError: list index out of range

## 1.3

In [65]:
import os
import requests
from bs4 import BeautifulSoup
from tqdm import tqdm
def Parse2(tsv, soup):
    animeTitle,animeType,(animeNumEpisode),(releaseDate),(endDate),(animeNumMembers),(animeScore),(animeUsers),(animeRank),(animePopularity),animeDescription,(animeRelated),(animeCharacters),(animeVoices),(animeStaff)="","","","","","","","","","","","","","",""

    left_col=soup.find('div', {'style':"width: 225px"})
    l_div=left_col.find_all("div",{"class":"spaceit_pad"})
    
    start=""
    end=""
    
    for i in range(len(l_div)):
        
        if "Type" in l_div[i].getText():
            animeType = l_div[i].getText().replace("Type:","").replace("\n","").strip()
            #print(animeType)
            
        if "Episodes" in l_div[i].getText():
            animeNumEpisode = (l_div[i].getText().replace("Episodes:","").replace("\n","").strip())
            if animeNumEpisode=="Unknown":
                animeNumEpisode=""
            #print(animeNumEpisode)
        
        if "Aired" in l_div[i].getText():
            if len(l_div[i].getText().replace("Aired:","").replace("to","").strip().split())==6:
                start=" ".join(l_div[i].getText().replace("Aired:","").replace(",","").replace("to","").strip().split()[0:3])
                end=" ".join(l_div[i].getText().replace("Aired:","").replace(",","").replace("to","").strip().split()[3:6])
                start=datetime.strptime(start,"%b %d %Y")
                end=datetime.strptime(end,"%b %d %Y")
            if len(l_div[i].getText().replace("Aired:","").replace(",","").replace("to","").strip().split())==3 and ("?" not in l_div[i].getText().replace("Aired:","").replace("to","").strip().split()):
                if len(l_div[i].getText().replace("Aired:","").replace(",","").replace("to","").strip().split()[1])!=len(l_div[i].getText().replace("Aired:","").replace(",","").replace("to","").strip().split()[2]):
                    
                    start=" ".join(l_div[i].getText().replace("Aired:","").replace(",","").replace("to","").strip().split()[0:3])
                    start=datetime.strptime(start,"%b %d %Y")
                else:
                    start=""
            else:
                start=end=""
            
            releaseDate=start
            endDate=end
            #print(releaseDate)
            #print(endDate)
            
        if ("Score" in l_div[i].getText()) and (i!=0) and ("Ookiku" not in l_div[i].getText()):
            if l_div[i].getText().replace("Score:","").replace(",","").split()[3]!="-":
                animeUsers = int(l_div[i].getText().replace("Score:","").replace(",","").split()[3])
                animeScore = float(l_div[i].getText().replace("Score:","").replace(",","").split()[0])
            #print(animeUsers)
            #print(animeScore)
        #if "Ranked" in l_div[i].getText():#19 >>>> conviene prenderlo da sopra il rank
            
        if "Popularity" in l_div[i].getText():  
            animePopularity = int(l_div[i].getText().replace("Popularity:","").strip().replace("#",""))
            #print(animePopularity)
    
        if "Members" in l_div[i].getText():
            animeNumMembers = int(l_div[i].getText().replace("Members:","").strip().replace(",",""))
            #print(animeNumMembers)
   
    
    description=soup.find("p",{"itemprop":"description"})
    animeDescription=description.getText().replace("\n","")
    #print(animeDescription)
    
    #THESE ARE THE SUGGESTED ONES
    #rel=[]
    #related=soup.find_all("li",{"class":"btn-anime","style":"width:90px"})
    #for i in range(len(related)):
    #    rel.append(related[i]["title"])
    #animeRelated = rel
    #print(animeRelated)
    rel=[]
    related=soup.find("table",{"class":"anime_detail_related_anime"})#.find_all("td",{"class":"borderClass"})
    if (related)!=None:
        #print(type(related),related)
        related=related.find_all("td",{"class":"borderClass"})
        for i in range(len(related)):
            if i%2!=0:
                rel.append(related[i].getText())
    animeRelated=rel
    
    chars=[]
    voices=[]
    staff=[]
    l=[]
    seed=soup.find_all("div",{"class":"detail-characters-list clearfix"})
    for i in range(len(seed)):
        if i==0:
            who=soup.find("div",{"class":"detail-characters-list clearfix"}).find_all("img")
            for i in range(len(who)):
                if i%2==0:
                    chars.append(",".join(who[i]["alt"].replace(" ","").split(",")))
                else:
                    voices.append(",".join(who[i]["alt"].replace(" ","").split(",")))
    
        if i==1:
            if (len(soup.find_all("div",{"class":"detail-characters-list clearfix"}))>1):
                for i in range(len(soup.find_all("div",{"class":"detail-characters-list clearfix"})[1].find_all("img"))):
                    staff.append(soup.find_all("div",{"class":"detail-characters-list clearfix"})[1].find_all("img")[i]["alt"])
                    staff.append((soup.find_all("div",{"class":"detail-characters-list clearfix"})[1].find_all("div", {"class":"spaceit_pad"}))[i].getText().strip())
                    
                for i in range(0,2*len(soup.find_all("div",{"class":"detail-characters-list clearfix"})[1].find_all("img")),2):
                    l.append(staff[i:i+2])
            
            
    animeCharacters = chars
    animeVoices = voices
    animeStaff = l   

    #print(animeStaff)
    
    
    animeTitle = soup.find("h1",{"class":"title-name h1_bold_none"}).getText()
    animeRank=soup.find("span",{"class":"numbers ranked"}).getText().replace("Ranked #","")
    if "N/A" not in animeRank:
        animeRank = int(animeRank)
    else:
        animeRank=""
    tsv.write("animeTitle"+"\t"+"animeType"+"\t"+"animeNumEpisode"+"\t"+"releaseDate"+"\t"+"endDate"+"\t"+"animeNumMembers"+"\t"+"animeScore"+"\t"+"animeUsers"+"\t"+"animeRank"+"\t"+"animePopularity"+"\t"+"animeDescription"+"\t"+"animeRelated"+"\t"+"animeCharacters"+"\t"+"animeVoices"+"\t"+"animeStaff"+"\n")

    tsv.write(animeTitle+"\t"+animeType+"\t"+str(animeNumEpisode)+"\t"+str(releaseDate)+"\t"+str(endDate)+"\t"+str(animeNumMembers)+"\t"+str(animeScore)+"\t"+str(animeUsers)+"\t"+str(animeRank)+"\t"+str(animePopularity)+"\t"+animeDescription+"\t"+str(animeRelated)+"\t"+str(animeCharacters)+"\t"+str(animeVoices)+"\t"+str(animeStaff))


In [70]:
#PARSING
import os
import requests
from bs4 import BeautifulSoup
from tqdm import tqdm
from datetime import datetime

for page in tqdm(range(0,382)):
    
    start=50*page
    
    for i in range(start,start+50):
        #Open the articles of the page 
        os.chdir("Page"+str(page+1))
        art=open("article"+str(i+1)+'.html',"r")
        os.chdir("..")
        os.chdir("TSV")
        tsv=open("anime_"+str(i+1)+".tsv","w")
        
        soup=BeautifulSoup(art.read(), "html.parser") 
        
        Parse2(tsv,soup)
        
        art.close()
        tsv.close()
        os.chdir("..")


100%|█████████▉| 381/382 [38:20<00:06,  6.04s/it] 


FileNotFoundError: [Errno 2] No such file or directory: 'article19084.html'

# Part 5

You consult for a personal trainer who has a back-to-back sequence of requests for appointments. A sequence of requests is of the form > 30, 40, 25, 50, 30, 20 where each number is the time that the person who makes the appointment wants to spend. You need to accept some requests, however you need a break between them, so you cannot accept two consecutive requests. For example, [30, 50, 20] is an acceptable solution (of duration 100), but [30, 40, 50, 20] is not, because 30 and 40 are two consecutive appointments. Your goal is to provide to the personal trainer a schedule that maximizes the total length of the accepted appointments. For example, in the previous instance, the optimal solution is [40, 50, 20], of total duration 110.

1. Write an algorithm that computes the acceptable solution with the longest possible duration.
1. Implement a program that given in input an instance in the form given above, gives the optimal solution.


In [76]:
#Implementation
def computeTime(A):
    l=len(A)
    if l<=1: return A[0]
    count=0
    index1=[]
    somma=0
    index2=[]
    for i in range(0,l,2):
        count = count + A[i]
        index1.append(i)
    for j in range(1,l,2):
        somma = somma + A[j]
        index2.append(j)
    if count>somma : return count, index1
    elif somma>count : return somma, index2
    else: return ("Both solutions are symmetrical")
        

In [77]:
A=[30, 40, 25, 50, 30, 20]
value,index=computeTime(A)
print(value,index)

110 [1, 3, 5]
